<a href="https://colab.research.google.com/github/A-Burnhard/OpenAI-Langchain-function-calling/blob/main/OpenAI_%26_Langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
!pip install langchain
!pip install openai
!pip install json
!pip install -U langchain-openai
!pip install pypdf
!pip install "langchain[docarray]"

ERROR: Could not find a version that satisfies the requirement json (from versions: none)
ERROR: No matching distribution found for json
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.3/215.3 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 7.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for hnswlib: filename=hnswlib-0.8.0-cp310-cp310-linux_x86_64.whl size=2287619 sha256=38198487ff2c9d3fedf8c34abf896479502965b71dfacc0892597d38e7ed1175
  Stored in directory: /root/.cache/pip/wheels/af/a9/3e/3e5d59ee41664eb31a4e6de67d1846f86d16d93c45f277c4e7
Successfully built hnswlib


In [10]:
#Langchain
from langchain_community.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain_community.document_loaders import TextLoader
from langchain.chains import RetrievalQA,  ConversationalRetrievalChain
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain_community.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain_community.document_loaders import TextLoader
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.tools import format_tool_to_openai_function, tool
#from langchain.globals import set_llm_cache, get_llm_cache
#New
#from langchain import HumanMessage, AIMessage, ChatMessage
from langchain_community.tools import format_tool_to_openai_function, YouTubeSearchTool, MoveFileTool
from langchain_community.vectorstores import Chroma
import openai
import datetime

In [2]:
import langchain
import openai
import json
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool
# Environment Variables
import os





In [4]:
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage, AIMessage, ChatMessage
from langchain.tools import format_tool_to_openai_function, YouTubeSearchTool, MoveFileTool
from langchain.agents import ConversationalAgent


In [56]:
openai.api_key =
openai_api_key = openai.api_key
llm = ChatOpenAI(model="gpt-4-0613", openai_api_key=openai.api_key)
current_date = datetime.datetime.now().date()

# Define the date after which the model should be set to "gpt-3.5-turbo"
target_date = datetime.date(2024, 6, 12)
# Set the model variable based on the current date
if current_date > target_date:
    llm_model = "gpt-4-0613"
else:
    llm_model = "gpt-4-0613"

#Internal Data Implementation
#- Load the data, split it into chunks, and embed it
loader = PyPDFLoader("data.pdf")
pages = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
documents = text_splitter.split_documents(pages)
embedding = OpenAIEmbeddings(openai_api_key=openai_api_key)
db = DocArrayInMemorySearch.from_documents(documents, embedding)
k=10
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": k})

#- Prompt Template
context= "receptionist"
template = """As a receptionist of Tech company called Nerasol.Use this context,reply to messages in three sentences or less :{context}.Question: {question} """

#- Define the QA model
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)
llm = ChatOpenAI(model_name="gpt-3.5-turbo-0301", openai_api_key=openai_api_key, temperature=0.9)
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=retriever,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT},
)


In [5]:
function_description = [{
    "name": "send_sms",
    "description": "Send an SMS to a specified phone number with a given message.",
    "parameters": {
        "type": "object",
        "properties": {
            "number": {
                "type": "string",
                "description": "The phone number of the recipient (e.g., '0550916600').",
            },
            "message": {
                "type": "string",
                "description": "The content of the SMS message.",
            },
        },
        "required": ["number", "message"],
    },
}
]

In [70]:
import requests  # Assuming you have the requests library installed

@tool
def send_sms(number, message):
    """
    Sends an SMS to the specified phone number with the given message.

    Parameters:
    - number (str): The phone number of the recipient (e.g., "0550916600").
    - message (str): The content of the SMS.

    Returns:
    - dict: A dictionary containing the result of the SMS sending operation.
    """
    # Construct the payload for the POST request
    payload = {
        "number": number,
        "message": message,
    }

    # Make a POST request to the specified URL
    url = "https://laws.adudor.com/api/fire-sms"
    response = requests.post(url, json=payload)

    # Return the result as a dictionary
    result = {
        "status_code": response.status_code,
        "response_content": response.json() if response.headers['content-type'] == 'application/json' else response.text,
    }

    return result

send_sms_tool = format_tool_to_openai_function(send_sms)



In [78]:
from langchain.agents import Tool

tools = [

    Tool(
        name='Knowledge Base',
        func=qa_chain.run,
        description=(
            'use this tool when answering gquestions as a receptionist of an IT firm called Nerasol '

        )
    ),
    Tool(
        name='Send SMS',
        func=send_sms,  # Assuming your `send_sms` function is defined
        description='Use this tool to send SMS, providing a phone number and message'

    )
]

In [79]:
# conversational memory
conversational_memory = ConversationBufferWindowMemory(
    memory_key='chat_history',
    k=5,
    return_messages=True
)
# retrieval qa chain
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever
)

from langchain.agents import initialize_agent

agent = initialize_agent(
    agent='chat-conversational-react-description',
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3,
    early_stopping_method='generate',
    memory=conversational_memory
)

In [80]:
query = 'send an sms to 0550916600 with a message that he should come i miss him'

In [81]:
agent(query)



> Entering new AgentExecutor chain...
{
    "action": "Send SMS",
    "action_input": "0550916600: he should come i miss him"
}

ValidationError: 1 validation error for send_smsSchemaSchema
message
  field required (type=value_error.missing)

In [ ]:
user_request = """ send an sms to 0550916600 tell him i need to see him"""

In [ ]:
first_response = llm.invoke([HumanMessage(content=user_request)],
                                      functions=function_description)
first_response

In [ ]:
first_response.additional_kwargs